# Table of Contents
1. [Step 1: Prepare Data for Consumption](#step1)
1. [Step 2: Merge Datasets and Add Rows](#step2)
1. [Step 3: Extract Data from Features](#step3)
1. [Step 4: Add weather data](#step4)


<a id="step1"></a>
# Step 1: Prepare Data for Consumption

## 1.1 Import Libraries

In [172]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.set_option('display.max_columns', None)
sns.set(style='white', context='notebook', palette='deep')

## 1.2 Load Data

In [173]:
df = pd.read_csv("./input/2018.csv")
df_extra = pd.read_csv("./input/df_with_neis.csv")

c:\projects\python\autotel-ml\venv\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\projects\python\autotel-ml\venv\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 1.2 Prepare Data for Cosumption

In [174]:
df_extra.drop(['vehicle_id','shiryun_cat','avg_dist','billing_minutes','orig_addr','dest_addr',
               'start_dt','end_dt','orig_addr_c','dest_addr_c','orig_x_merc',
               'orig_y_merc','dest_x_merc','dest_y_merc','start_hour','start_dayofweek','start_month'],axis=1,inplace=True)

In [175]:
df = df.rename({'מספר רכב': 'Car id', 'מספר הזמנה': 'Order number','קטגוריית שריון':'Category',
                 'ממוצע מרחק לנסיעה':'Avg distance','דקות לחיוב':'Billing minutes','כתובת התחלה':'Start Address',
                 'כתובת סיום':'Dest Address','תאריך ושעה התחלת שריון':'Start time','תאריך ושעה סיום הזמנה':'End time'},
                axis=1) 

In [176]:
df_extra = df_extra.rename({'order_id': 'Order number'},axis=1) 

<a id="step2"></a>
# Step 2: Merge Datasets and Add Rows

## 2.1 Merge Datasets

In [177]:
df = pd.merge(df, df_extra, on=['Order number'], how='left')

## 2.2 Create New Rows
creating new rows from destination data (Dest Address, End time, dest_coord, dest_x, dest_y, dest_nei)

In [178]:
df_temp = df.copy()

In [179]:
df.drop(["Dest Address","End time","dest_coord","dest_x","dest_y","dest_nei"],axis=1,inplace=True)
df_temp.drop(["Start Address","Start time","orig_coord","orig_x","orig_y","orig_nei"],axis=1,inplace=True)

In [180]:
df = df.rename({"Start Address":"Address","Start time": "Time","orig_coord":"Coords","orig_x":"Lat","orig_y":"Lon","orig_nei":"neighborhood",},axis=1)
df_temp = df_temp.rename({"Dest Address":"Address","End time": "Time","dest_coord":"Coords","dest_x":"Lat","dest_y":"Lon","dest_nei":"neighborhood"},axis=1)

In [181]:
df = pd.concat([df,df_temp])

In [182]:
df.sort_values(["Order number","Time"],axis=0,inplace=True)

### [Delete] check for null and missing values

In [183]:
# Fill empty and NaNs values with NaN
df = df.fillna(np.nan)

In [209]:
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136331 entries, 0 to 136330
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Order number  136331 non-null  int64  
 1   kmh           134214 non-null  float64
 2   orig_coord    136331 non-null  object 
 3   dest_coord    136331 non-null  object 
 4   orig_x        136331 non-null  float64
 5   orig_y        136331 non-null  float64
 6   dest_x        136331 non-null  float64
 7   dest_y        136331 non-null  float64
 8   orig_nei      123860 non-null  object 
 9   dest_nei      123860 non-null  object 
dtypes: float64(5), int64(1), object(4)
memory usage: 8.3+ MB


Order number        0
kmh              2117
orig_coord          0
dest_coord          0
orig_x              0
orig_y              0
dest_x              0
dest_y              0
orig_nei        12471
dest_nei        12471
dtype: int64

<a id="step3"></a>
# Step 3: Extract Data from Features

### Splitting address to country, city, street

In [185]:
split_col = df["Address"].str.split(",", expand=True)
split_col[:3]

,0,1,2,3
0,גורדון 86-88,תל אביב יפו,ישראל,None
0,הרב רובינשטיין 61,תל אביב יפו,ישראל,None
1,יעקב אפטר 3,תל אביב יפו,ישראל,None


In [186]:
df["Street"] = split_col[0]
df["City"] = split_col[1]
df["Country"] = split_col[2]

#### extracting street without numbers

In [187]:
df["Street_c"] = df['Street'].str.replace(r'(\d+)(-(\d+))*', r'')

#### extract clean address for google api use

In [188]:
df['Address_c'] = df['Address'].str.replace(r'(\d+)-(\d+)', r'\1')

### Extract date and time

In [189]:
df["Time"] = pd.to_datetime(df["Time"])

In [190]:
df["Date"] = df["Time"].dt.date
# df["time"] = df["Time"].dt.time
df["Year"] = df["Time"].dt.year
df["Month"] = df["Time"].dt.month
df["Day"] = df["Time"].dt.day
df["Hour"] = df["Time"].dt.hour
df["Minute"] = df["Time"].dt.minute

In [191]:
df.head()

,Car id,Order number,Category,Avg distance,Billing minutes,Address,Time,kmh,Coords,Lat,Lon,neighborhood,Street,City,Country,Street_c,Address_c,Date,Year,Month,Day,Hour,Minute
0,46-525-81,60386,שריון חינמי,8,13,"גורדון 86-88, תל אביב יפו, ישראל",2017-10-23 00:08:00,37.151703,"(32.0807428, 34.7792228)",32.080743,34.779223,הצפון הישן-החלק הדרומי,גורדון 86-88,תל אביב יפו,ישראל,גורדון,"גורדון 86, תל אביב יפו, ישראל",2017-10-23,2017.0,10.0,23.0,0.0,8.0
0,46-525-81,60386,שריון חינמי,8,13,"הרב רובינשטיין 61, תל אביב יפו, ישראל",2017-10-23 00:31:00,37.151703,"(32.0357011, 34.7581719)",32.035701,34.758172,(יפו ד' (גבעת התמרים,הרב רובינשטיין 61,תל אביב יפו,ישראל,הרב רובינשטיין,"הרב רובינשטיין 61, תל אביב יפו, ישראל",2017-10-23,2017.0,10.0,23.0,0.0,31.0
1,48-437-81,60387,שריון חינמי,3,8,"יעקב אפטר 3, תל אביב יפו, ישראל",2017-10-23 00:09:00,21.556886,"(32.1050698, 34.7892274)",32.105070,34.789227,'תכנית ל,יעקב אפטר 3,תל אביב יפו,ישראל,יעקב אפטר,"יעקב אפטר 3, תל אביב יפו, ישראל",2017-10-23,2017.0,10.0,23.0,0.0,9.0
1,48-437-81,60387,שריון חינמי,3,8,"ז'בוטינסקי 10, תל אביב יפו, ישראל",2017-10-23 00:21:00,21.556886,"(32.0895831, 34.7734306)",32.089583,34.773431,הצפון הישן - החלק הצפוני,ז'בוטינסקי 10,תל אביב יפו,ישראל,ז'בוטינסקי,"ז'בוטינסקי 10, תל אביב יפו, ישראל",2017-10-23,2017.0,10.0,23.0,0.0,21.0
2,63-887-81,60388,שריון חינמי,0,2,"רופין 32-34, תל אביב יפו, ישראל",2017-10-23 00:10:00,0.000000,"(32.0836176, 34.7710507)",32.083618,34.771051,הצפון הישן-החלק הדרומי,רופין 32-34,תל אביב יפו,ישראל,רופין,"רופין 32, תל אביב יפו, ישראל",2017-10-23,2017.0,10.0,23.0,0.0,10.0


<a id="step4"></a>
# Step 4: Add weather data

In [192]:
weather = pd.read_csv("./input/ims_data_2017_2018.csv",na_values={
    "טמפרטורה(C°)": ["-"],
    "טמפרטורת מקסימום(C°)": ["-"],
    "טמפרטורת מינימום(C°)": ["-"],
    "לחות יחסית(%)": ["-"],
    "מהירות הרוח(m/s)": ["-"],
    "כיוון הרוח(מעלות)": ["-"],
    "מהירות רוח דקתית מקסימלית(m/s)": ["-"],
    "מהירות רוח 10 דקתית מקסימלית(m/s)": ["-"]
},dtype={
    "טמפרטורה(C°)": np.float64,
    "טמפרטורת מקסימום(C°)":np.float64,
    "טמפרטורת מינימום(C°)":np.float64,
    "לחות יחסית(%)": np.float64,
    "מהירות הרוח(m/s)":np.float64,
    'כמות גשם(מ"מ)': np.float64,
    "מהירות הרוח(m/s)": np.float64,
    "כיוון הרוח(מעלות)": np.float64,
    "מהירות רוח דקתית מקסימלית(m/s)": np.float64,
    "מהירות רוח 10 דקתית מקסימלית(m/s)": np.float64
})

In [193]:
weather = weather.rename({"שם תחנה":"station","תאריך":"Date","שעה- LST":"Time - LST","טמפרטורה(C°)":"Temprature",
                          "טמפרטורת מקסימום(C°)":"Max Temprature","טמפרטורת מינימום(C°)":"Min Temprature","לחות יחסית(%)":"Relative Humidity",
                          'כמות גשם(מ"מ)':"Amount of Rain","מהירות הרוח(m/s)":"Wind Speed","כיוון הרוח(מעלות)":"Wind Direction",
                          "מהירות רוח דקתית מקסימלית(m/s)":"Max Wind Speed per Min","מהירות רוח 10 דקתית מקסימלית(m/s)":"Max Win Speed per 10 Min"}
                         ,axis=1)

In [194]:
weather["Time - LST"] = pd.to_datetime(weather["Time - LST"])
weather["Date"] = pd.to_datetime(weather["Date"])

In [195]:
weather["Hour"] = weather["Time - LST"].dt.hour.astype(float)
weather["Date"] = weather["Date"].dt.date

In [196]:
weather.head()

,station,Date,Time - LST,Temprature,Max Temprature,Min Temprature,Relative Humidity,Amount of Rain,Wind Speed,Wind Direction,Max Wind Speed per Min,Max Win Speed per 10 Min,Hour
0,תל-אביב חוף ...,2017-01-01,2020-03-25 00:00:00,10.4,10.5,10.3,84.0,0.0,2.6,114.0,2.8,2.7,0.0
1,תל-אביב חוף ...,2017-01-01,2020-03-25 00:10:00,10.3,10.3,10.2,84.0,0.0,2.5,115.0,2.9,2.6,0.0
2,תל-אביב חוף ...,2017-01-01,2020-03-25 00:20:00,10.2,10.2,10.2,84.0,0.0,2.3,114.0,2.6,2.6,0.0
3,תל-אביב חוף ...,2017-01-01,2020-03-25 00:30:00,10.1,10.2,10.1,83.0,0.0,1.8,112.0,2.4,2.3,0.0
4,תל-אביב חוף ...,2017-01-01,2020-03-25 00:40:00,10.1,10.1,10.1,83.0,0.0,1.8,108.0,2.3,1.8,0.0


In [197]:
weather_hour_grouped = weather.groupby(["Date","Hour"]).agg({
        'Temprature':'mean', 'Max Temprature':'max','Min Temprature':'min',
        'Relative Humidity':'mean','Amount of Rain':'mean','Wind Speed':'mean',
        'Wind Direction':'mean','Max Wind Speed per Min':'max','Max Win Speed per 10 Min':'max'}).reset_index()

In [198]:
df = pd.merge(df,weather_hour_grouped,on=["Date","Hour"],how='left')

## Visualization

In [65]:
counts = df['neighborhood'].value_counts()
counts_more_then = df[df['neighborhood'].isin(counts[counts > 1000].index)]

In [165]:
tmp = df.groupby(['Year', 'Month']).size().reset_index(name='Freq')